# Methods2Test Meta Dataset

## Golden Commits

In [ ]:
from datasets import Dataset
import json
import pandas as pd
from datasets import DatasetDict, Features, Sequence, Value

ddict = DatasetDict()
from datasets import DatasetDict, Features, Sequence, Value

feat = Features(
    {
        "id": Value(dtype="string"),
        "commit": Value(dtype="string"),
        "candidates": Value(dtype="string"),
    }
)

for split in ["validation", "test"]:
    def gen_ds():
        with open(f".data/commits_{split}.jsonl") as f:
            for line in f:
                yield json.loads(line)
            
    ds = Dataset.from_generator(generator=gen_ds, features=feat)
    ddict[split] = ds



Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
#ddict.push_to_hub("andstor/methods2test_meta", config_name="commit_candidates", private=False, max_shard_size="250MB")

In [ ]:
ddict = ddict.remove_columns(column_names=["candidates"])

In [ ]:
#ddict.push_to_hub("andstor/methods2test_meta", config_name="golden_commit", private=False, max_shard_size="250MB")

## Test Statuses

In [1]:
from datasets import load_dataset

dataset_meta = load_dataset("andstor/methods2test_meta", "golden_commit", split="test")

In [6]:
from datasets import Dataset
import json
import pandas as pd
from datasets import DatasetDict, Features, Sequence, Value


feat = Features(
    {
    'id': Value(dtype='string', id=None),
    'build_tool': Value(dtype='string', id=None),
    'status': Value(dtype='string', id=None)
    }
)

runnable_data = []
with open(f"./output/runnable.jsonl") as f:
    for line in f:
        runnable_data.append(json.loads(line))
            
runnable_df = pd.DataFrame(runnable_data).set_index("id")
runnable_df["repo_id"] = runnable_df.index.str.split("_").str[0]
runnable_df["test_id"] = runnable_df.index.str.split("_").str[1]
runnable_df = runnable_df.reset_index().set_index("repo_id")

def gen_ds():
    for row in dataset_meta:
        repo_id = row["id"].split("_")[0]
    
        if repo_id in runnable_df.index:
            runnable_row = runnable_df.loc[repo_id]
            row["build_tool"] = runnable_row["build_tool"]
            row["status"] = runnable_row["status"]
            yield row
        
        else:
            yield row

ds = Dataset.from_generator(generator=gen_ds, features=feat, split="test")

Generating test split: 0 examples [00:00, ? examples/s]

In [7]:
ds.to_pandas()["status"].value_counts()

status
build error    34953
success         7141
error            255
skipped            6
Name: count, dtype: int64

In [ ]:
#ds.push_to_hub("andstor/methods2test_meta", config_name="test_status", private=False, max_shard_size="250MB")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/75 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/andstor/methods2test_meta/commit/e57ba2b686510d2c857936c97d32c3f3a32a81b3', commit_message='Upload dataset', commit_description='', oid='e57ba2b686510d2c857936c97d32c3f3a32a81b3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/andstor/methods2test_meta', endpoint='https://huggingface.co', repo_type='dataset', repo_id='andstor/methods2test_meta'), pr_revision=None, pr_num=None)